# Test benchmark

## Setup

In [3]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote =
{__version__} {'(cython)' if is_compiled() else ''}
""")


== Python ==
3.10.4 (main, Mar 23 2022, 20:25:24) [GCC 11.3.0]

== Antidote =
2.0.0b1.dev3+g442bca8.d19800101 



In [4]:
cat /proc/cpuinfo | grep 'model name' | head -n 1

model name	: AMD Ryzen 7 PRO 4750U with Radeon Graphics


## Benchmarks

### Utilities

In [5]:
from antidote import world, interface, implements, lazy, injectable, const

services = []
lazies = []
interfaces = []
constants = []

def add_service(i: int):
    services.append(injectable(type(f"Service{i}", (object,), {})))

def add_lazy(i: int):
    Output = type(f"Ouput{i}", (object,), {})
    def f() -> Output:
        return Output()
    lazies.append(lazy.value(f))

def add_const(i: int):
    constants.append(const(i))

def add_implementation(i: int):
    Interface = interface(type(f"Interface{i}", (object,), {}))
    implements(Interface)(type(f"Impl{i}", (Interface,), {}))
    interfaces.append(Interface)

def generate_dependencies(n: int):
    k = n // 4
    for i in range(k):
        add_service(i)
        add_lazy(i + 1)
        add_const(i + 2)
        add_implementation(i + 3)

    # Instantiating all dependencies

    for dep in services:
        world[dep]

    for dep in lazies:
        world[dep]

    for dep in interfaces:
        world[dep]

    for dep in constants:
        world[dep]


def ref():
    pass

def empty():
    with world.test.empty():
        pass

def new():
    with world.test.new():
        pass

def clone():
    with world.test.clone():
        pass

def copy():
    with world.test.copy():
        pass

def benchmark():
    print("Reference (nothing)")
    %timeit ref()
    print("\nworld.test.empty()")
    %timeit empty()
    print("\nworld.test.new()")
    %timeit new()
    print("\nworld.test.clone()")
    %timeit clone()
    print("\nworld.test.copy()")
    %timeit copy()



### Isolation
#### Imitating a medium project

Creating **40** different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [8]:
generate_dependencies(n = 40)

In [9]:
benchmark()

Reference (nothing)
66.7 ns ± 7.52 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
world.test.empty()
40.8 µs ± 327 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.new()
161 µs ± 531 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.clone()
90.9 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.copy()
94.7 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### Imitating a big project

Creating **400** different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [10]:
generate_dependencies(n = 360)

In [11]:
benchmark()

Reference (nothing)
72.8 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
world.test.empty()
40.8 µs ± 771 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.new()
161 µs ± 973 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.clone()
235 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

world.test.copy()
257 µs ± 662 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Imitating a REALLY big project

Creating **4000** different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [12]:
generate_dependencies(n = 3600)

In [13]:
benchmark()

Reference (nothing)
63.2 ns ± 0.653 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
world.test.empty()
43.6 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.new()
162 µs ± 544 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

world.test.clone()
1.9 ms ± 6.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

world.test.copy()
2.18 ms ± 68.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Override

In [14]:
x = object()
with world.test.clone() as overrides:
    %timeit overrides[services[0]] = x

3.05 µs ± 903 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [1]:
%%scalene --reduced-profile
x = object()

def static():
    return x

with world.test.clone() as overrides:
    for _ in range(10000):
        overrides.factory(services[0])(static)

UsageError: Cell magic `%%scalene` not found.


In [2]:
%load_ext scalene


Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version.
